<a href="https://colab.research.google.com/github/jmhuer/shift_invariant_dictionary_learning/blob/main/midi_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
!pip install magenta

!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -q pyfluidsynth


Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package pyfluidsynth


In [29]:

# Allow python to pick up the newly-installed fluidsynth lib.
# This is only needed for the hosted Colab environment.
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library
  
print('Importing software libraries...')

import copy, warnings, librosa, numpy as np
warnings.filterwarnings("ignore", category=DeprecationWarning)


Importing software libraries...


In [55]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Colab/Notebook specific stuff
import IPython.display
from IPython.display import Audio
from google.colab import files

# Magenta specific stuff
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel
from magenta.models.music_vae import data
import note_seq
from note_seq import midi_synth
from note_seq.sequences_lib import concatenate_sequences
from magenta.models.music_vae import configs
dc_4bar = configs.CONFIG_MAP['groovae_4bar'].data_converter


# tfds works in both Eager and Graph modes
# tf.enable_eager_execution()


# Quick and dirty way to set midi files to be recognized as drums
def set_to_drums(ns):
  for n in ns.notes:
    n.instrument=9
    n.is_drum = True

def is_4_4(s):
  ts = s.time_signatures[0]
  return (ts.numerator == 4 and ts.denominator ==4)

# Calculate quantization steps but do not remove microtiming
def quantize(s, steps_per_quarter=4):
  return note_seq.sequences_lib.quantize_note_sequence(s,steps_per_quarter)


def download(note_sequence, filename):
  note_seq.sequence_proto_to_midi_file(note_sequence, filename)
  files.download(filename)


# If a sequence has notes at time before 0.0, scootch them up to 0
def start_notes_at_0(s):
  for n in s.notes:
    if n.start_time < 0:
      n.end_time -= n.start_time
      n.start_time = 0
  return s


def preprocess_4bar(s):
  return dc_4bar.from_tensors(dc_4bar.to_tensors(s).outputs)[0]


def tensor_4bar(s):
  return dc_4bar.to_tensors(s).outputs[0]



# dataset_4bar = tfds.as_numpy(tfds.load(
#     name="groove/4bar-midionly",
#     split=tfds.Split.VALIDATION,
#     try_gcs=True))



# dev_sequences_4bar = [quantize(note_seq.midi_to_note_sequence(features["midi"])) for features in dataset_4bar]
# _ = [set_to_drums(s) for s in dev_sequences_4bar]
# dev_sequences_4bar = [s for s in dev_sequences_4bar if is_4_4(s) and len(s.notes) > 0 and s.notes[-1].quantized_end_step > note_seq.steps_per_bar_in_quantized_sequence(s)]



# download(dev_sequences_4bar[1], "huhb.mid")

# print(preprocess_4bar(dev_sequences_4bar[1]))
# print("nrw\n\n\n", dev_sequences_4bar[1])

print(tensor_4bar(dev_sequences_4bar[1]).shape)


(64, 27)
